In [ ]:
import fastplotlib as fpl
import numpy as np
import zmq

In [ ]:
context = zmq.Context()

# create subscriber
sub = context.socket(zmq.SUB)
sub.setsockopt(zmq.SUBSCRIBE, b"")

# keep only the most recent message
sub.setsockopt(zmq.CONFLATE, 1)

# publisher address and port
sub.connect("tcp://127.0.0.1:5555")

In [ ]:
def get_bytes():
    """
    Gets the bytes from the publisher
    """
    try:
        b = sub.recv(zmq.NOBLOCK)
    except zmq.Again:
        pass
    else:
        return b

    return None

In [ ]:
fig = fpl.Figure()

# initialize some data, must be of same dtype and shape as data sent by publisher
data = np.random.rand(512, 512).astype(np.float32)
fig[0, 0].add_image(data, name="image")


def update_frame(subplot):
    # recieve bytes
    b = get_bytes()

    if b is not None:
        # numpy array from bytes, MUST specify dtype and make sure it matches what you sent
        a = np.frombuffer(b, dtype=np.float32).reshape(512, 512)

        # set graphic data
        subplot["image"].data = a


fig[0, 0].add_animations(update_frame)
fig.show()